In [ ]:
'''
   "51":  "Right lateral ventricle",
   "52":  "Left lateral ventricle",
   "4":  "third ventricle",
   "11":  "fourth ventricle"
   
   "6": acquedukt
'''

In [ ]:
import glob
import nibabel as nib
import numpy as np
import os
import cupy as cp
import pandas as pd
import re
import time
import seaborn as sns
from joblib import Parallel, delayed

In [ ]:
def dice(f1, f2,n):
    arr1 = nib.load(f1).get_fdata()
    arr2 = nib.load(f2).get_fdata()
    arr1 = np.where(arr1==n,1,0)
    arr2 = np.where(arr2==n,1,0)
    return (arr1 & arr2).sum()*2/(arr1.sum() + arr2.sum())

In [ ]:
manual_mask = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg_manual/sub-*/ses-01/anat/sub-V*_ses-01_space-T1w_desc-ventricle-mask.nii.gz'
gt = glob.glob(manual_mask)

In [ ]:
#gt

In [ ]:
subids = [i.split('/')[-4] for i in gt]


In [ ]:
len(subids)

In [ ]:
subids

In [ ]:
cnn = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg/{subid}/{subid}_n4_mni_strip_seg_inverse.nii.gz'
gt = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg_manual/{subid}/ses-01/anat/{subid}_ses-01_space-T1w_desc-ventricle-mask.nii.gz'

In [ ]:
roinums = [51, 52, 4, 11]

In [ ]:
cnns =[cnn.format(subid=i) for i in subids]
gtss =[gt.format(subid=i) for i in subids]

In [ ]:
cnns =[nib.load(i) for i in cnns]
gtss =[nib.load(i) for i in gtss]

In [ ]:
start = time.time()
d = dict()
for i in subids:
    try:
        d[i] = Parallel(n_jobs=4)(delayed(dice)(cnn.format(i),gt.format(i),n) for n in roinums)
        print('made: ', i)
    except:
        print('skipped: ', i)
stop = time.time()
print(stop-start)
#748.1604852676392s